# Legislación

## Contexto
Los cerros que rodean la planicie central de la cuenca de Santiago imponen fuertes restricciones a la circulación de vientos y, por ende, a la renovación del aire al interior de la cuenca. Por ello, en épocas de estabilidad atmosférica los contaminantes quedan atrapados dentro de la cuenca que alberga a la ciudad de Santiago.

Por esta situación en particular santiago se ha visto varias veces enfrentado con una contaminación muy alta, llegando a ser declarada zona saturada por Ozono, Material Particulado, Monoxido de Carbono y Dioxido de Nitrógeno en 1996, el año anterior se ordena la creación del _Plan de Prevención y Descontaminación Atmosférica_ de la Región Metropolitana, que desde 1998 norma con el objetivo de:

El PPDA tiene por objetivo cumplir con las normas de calidad de aire cuyo propósito principal es la protección de la salud de la población. Ésta se encuentra habitualmente expuesta a concentraciones de contaminantes que afectan su bienestar físico y su calidad de vida en general. -- DS N°16 de 1998, Minsegpres, CONAMA

Con esto se realizan desde el mismo año mediciones diarias de distintos agentes contaminantes para evaluar tanto la situación actual del aire en santiago como para medir el rendimiento del PPDA.

| Fecha        | Tipo    | Nombre                                 |
|--------------|---------|----------------------------------------|
| 26-oct-95    | Decreto | Procedimiento PPDA                     |
| 01-ago-96    | Decreto | Zona saturada por O3 MP10 MP2,5 CO NO2 |
| 06-jun-98    | Decreto | Establece PPDA                         |
| 29-ene-04    | Decreto | Actualiza PPDA                         |
| 16-abr-10    | Decreto | Actualiza PPDA                         |
| 15-nov-14    | Decreto | Zona Saturada MP2,5                    |
| 24-nov-17    | Decreto | Establece PPDA                         |


Aquí intentaremos responder si estas legislaciones y regulaciones tienen algún efecto real sobre la disminución de la contaminación en la cuenca de Santiago.

In [1]:
# Imports used on the script

import matplotlib
matplotlib.use('tkagg')

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pandas as pd

In [2]:
# Import climate data
df = pd.read_csv("C:/Users/basty/Documents/intro_mineria_dcc_2019/Proyecto/ClimaYEmisionesData/DatosHito1CleanWithDates.csv")
df['Fecha'] = pd.to_datetime(df['Fecha'])    
df['Fecha_rel'] = (df['Fecha'] - df['Fecha'].min())  / np.timedelta64(1,'D')

# Example of how the table looks
df.head()

,Fecha,anno,mes,dia,O3,MP10,CO,SO2,MP25,NO2,Tmedia,Tmax,Precipitaciones,Fecha_rel
0,1997-04-02,1997,4,2,18.8750,111.565,NaN,NaN,NaN,NaN,17.6,27.5,0.0,0.0
1,1997-04-03,1997,4,3,14.2083,141.333,NaN,NaN,NaN,NaN,16.0,27.2,0.0,1.0
2,1997-04-04,1997,4,4,30.4545,156.950,NaN,NaN,NaN,NaN,17.6,31.1,0.0,2.0
3,1997-04-05,1997,4,5,NaN,0.000,NaN,NaN,NaN,NaN,16.9,29.6,0.0,3.0
4,1997-04-06,1997,4,6,NaN,0.000,NaN,NaN,NaN,NaN,15.2,25.6,0.0,4.0


In [3]:
# We split the database on periods according to different versions of PPDA program

prePPDA = df[(df['Fecha']<'1998/06/06')]

primerPPDA = df[(df['Fecha']>='1998/06/06') & (df['Fecha']<='2004/01/29')]

segundoPPDA = df[(df['Fecha']>='2004/01/29') & (df['Fecha']<='2010/04/16')]

# This one is a complete new DS that declares the RM a MP2.5 Saturation Zone
tercerPPDA = df[(df['Fecha']>='2010/04/16') & (df['Fecha']<='2017/11/24')]

cuartoPPDA = df[(df['Fecha']>='2017/11/24')]

post98 = df[df['Fecha']>='1998/06/06']

post04 = df[df['Fecha']>='2004/01/29']

post10 = df[df['Fecha']>='2010/04/16']

post17 = df[df['Fecha']>='2017/11/24']

cuartoPPDA.head()

,Fecha,anno,mes,dia,O3,MP10,CO,SO2,MP25,NO2,Tmedia,Tmax,Precipitaciones,Fecha_rel
7541,2017-11-24,2017,11,24,17.0,49.0,0.36,NaN,13.0,23.73,16.5,25.4,0.0,7541.0
7542,2017-11-25,2017,11,25,28.0,61.0,0.49,NaN,18.0,27.48,20.9,32.0,0.0,7542.0
7543,2017-11-26,2017,11,26,22.0,61.0,0.52,NaN,16.0,22.65,20.0,29.1,0.0,7543.0
7544,2017-11-27,2017,11,27,21.0,53.0,0.52,NaN,17.0,27.03,19.1,29.2,0.0,7544.0
7545,2017-11-28,2017,11,28,14.0,50.0,0.52,NaN,17.0,25.21,15.4,23.7,0.0,7545.0


In [4]:
# Plot outputs
plt.scatter(df['Fecha'], df['NO2'],  color='black')
plt.title('Proyecciones de NO2')
plt.xlabel('Fecha')
plt.ylabel('μg/m3')
plt.xticks(())
plt.yticks(np.arange(0, 301, step=20))

import matplotlib.dates as mdates

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=365))
plt.gcf().autofmt_xdate()

c:\users\basty\appdata\local\programs\python\python37-32\lib\site-packages\pandas\plotting\_matplotlib\converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


In [5]:
model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                  ('linear', LinearRegression(fit_intercept=False))])

# Create our imputer to replace missing values with the mean
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(df['NO2'].values.reshape(len(df['NO2']),1))

X_train = prePPDA['Fecha_rel'].values.reshape(len(prePPDA['Fecha_rel']),1)

Y_train = imp.transform(prePPDA['NO2'].values.reshape(len(prePPDA['NO2']),1))

X_test = post98['Fecha_rel'].values.reshape(len(post98['Fecha_rel']),1)

model = model.fit(X_train,Y_train)
model.named_steps['linear'].coef_
plt.plot(post98['Fecha'], model.predict(X_test) , color='blue', linewidth=3)

In [6]:
X_train = primerPPDA['Fecha_rel'].values.reshape(len(primerPPDA['Fecha_rel']),1)

Y_train = imp.transform(primerPPDA['NO2'].values.reshape(len(primerPPDA['NO2']),1))

X_test = post04['Fecha_rel'].values.reshape(len(post04['Fecha_rel']),1)

model = model.fit(X_train,Y_train)
model.named_steps['linear'].coef_
plt.plot(post04['Fecha'], model.predict(X_test) , color='blue', linewidth=3)

In [7]:
X_train = segundoPPDA['Fecha_rel'].values.reshape(len(segundoPPDA['Fecha_rel']),1)

Y_train = imp.transform(segundoPPDA['NO2'].values.reshape(len(segundoPPDA['NO2']),1))

X_test = post10['Fecha_rel'].values.reshape(len(post10['Fecha_rel']),1)

model = model.fit(X_train,Y_train)
model.named_steps['linear'].coef_
plt.plot(post10['Fecha'], model.predict(X_test) , color='blue', linewidth=3)

In [8]:
# Create linear regression object
#regr = LinearRegression()

# Train the model using the training sets
#regr.fit(prePPDA['Fecha_rel'].values.reshape(len(prePPDA['Fecha_rel']),1), imp.transform(prePPDA['MP10'].values.reshape(len(prePPDA['MP10']),1)))

# Plot outputs
# plt.plot(primerPPDA['Fecha'], regr.predict(X_test) , color='red', linewidth=3)

In [9]:
X_train = tercerPPDA['Fecha_rel'].values.reshape(len(tercerPPDA['Fecha_rel']),1)

Y_train = imp.transform(tercerPPDA['NO2'].values.reshape(len(tercerPPDA['NO2']),1))

X_test = post17['Fecha_rel'].values.reshape(len(post17['Fecha_rel']),1)

model = model.fit(X_train,Y_train)
model.named_steps['linear'].coef_
plt.plot(post17['Fecha'], model.predict(X_test) , color='blue', linewidth=3)

In [10]:
plt.show()

In [20]:
# Caracteristicas de la wea

df.cov()

,anno,mes,dia,O3,MP10,CO,SO2,MP25,NO2,Tmedia,Tmax,Precipitaciones,Fecha_rel
anno,41.914465,-1.221341,-0.174725,-7.995907,-26.790036,-1.391009,-18.908833,-19.047001,6.545927,0.762155,3.345686,-1.586188,1.527187e+04
mes,-1.221341,11.724317,0.298808,-0.509329,-18.959958,-0.158501,0.355777,-5.692477,-2.617360,-3.180115,-4.131131,0.268161,-8.889411e+01
dia,-0.174725,0.298808,77.374795,-0.267079,0.416576,0.043920,2.944094,1.250779,0.936283,-0.159258,-0.270742,-0.022721,2.249902e+01
O3,-7.995907,-0.509329,-0.267079,69.952755,-77.035655,-4.074538,-9.000962,-65.080376,-57.709444,33.613154,43.891076,-8.061287,-2.933847e+03
MP10,-26.790036,-18.959958,0.416576,-77.035655,1395.805194,28.256617,177.634005,536.946864,328.641827,-52.593195,-16.302251,-24.511796,-1.037059e+04
CO,-1.391009,-0.158501,0.043920,-4.074538,28.256617,1.028367,5.734085,13.157637,6.301732,-2.677759,-2.376364,0.120598,-5.131025e+02
SO2,-18.908833,0.355777,2.944094,-9.000962,177.634005,5.734085,63.473488,69.987576,22.798371,-9.414234,-5.644161,-1.092591,-6.893730e+03
MP25,-19.047001,-5.692477,1.250779,-65.080376,536.946864,13.157637,69.987576,347.108824,152.215379,-43.478934,-36.139517,-4.989201,-7.133293e+03
NO2,6.545927,-2.617360,0.936283,-57.709444,328.641827,6.301732,22.798371,152.215379,191.797393,-36.802365,-26.661257,-0.420446,2.307969e+03
Tmedia,0.762155,-3.180115,-0.159258,33.613154,-52.593195,-2.677759,-9.414234,-43.478934,-36.802365,25.442366,29.818766,-3.479991,1.830538e+02


In [19]:
columnas = ['anno', 'mes', 'dia', 'O3', 'MP10', 'CO', 'SO2', 'MP25', 'NO2', 'Tmedia', 'Tmax', 'Precipitaciones']

for col in columnas:
    print(df[col].describe())
    print("\n")

count    8179.000000
mean     2007.947549
std         6.474138
min      1997.000000
25%      2002.000000
50%      2008.000000
75%      2014.000000
max      2019.000000
Name: anno, dtype: float64


count    8179.000000
mean        6.512532
std         3.424079
min         1.000000
25%         4.000000
50%         7.000000
75%         9.000000
max        12.000000
Name: mes, dtype: float64


count    8179.000000
mean       15.720504
std         8.796294
min         1.000000
25%         8.000000
50%        16.000000
75%        23.000000
max        31.000000
Name: dia, dtype: float64


count    7922.000000
mean       15.393299
std         8.363776
min         0.712376
25%         7.916660
50%        16.050400
75%        22.043050
max        47.466200
Name: O3, dtype: float64


count    8153.000000
mean       72.730928
std        37.360476
min         0.000000
25%        48.000000
50%        64.083300
75%        90.916600
max       295.750000
Name: MP10, dtype: float64


count    7899.00000